# Домашнее задание HW 4

## Задание HW 4

Домашнее задание состоит из нескольких блоков.


### Эксперименты в ipynb ноутбуках (11 баллов)
- Необходимо будет перебрать $N$ моделей $(N \geq 2)$ матричной факторизации и перебрать у них $K$ гиперпараметров $(K \geq 2)$ **(5 баллов)**
    - Для перебора гиперпараметров можно использовать [`Optuna`](https://github.com/optuna/optuna), [`Hyperopt`](https://github.com/hyperopt/hyperopt)
- Воспользоваться методом приближенного поиска соседей для выдачи рекомендаций. **(2 балла)**
    - Можно использовать любые удобные: [`Annoy`](https://github.com/spotify/annoy), [`nmslib`](https://github.com/nmslib/nmslib) и.т.д
- Добавить 3 "аватаров" (искусственных пользователей) и посмотреть рекомендации итоговой модели на них. Объяснить почему добавили именно таких пользователей. **(2 балла)**
- Придумать как можно обработать рекомендации для холодных пользователей. **(2 балла)**

Примечание: за невоспроизводимый код в ноутбуках (например, нарушен порядок выполнения ячеек, вызываются переменные, которые нигде не были объявлены ранее и.т.п) будут штрафы на усмотрение проверяющего.


### Реализация итоговой модели в сервисе (9 баллов)
- Пробитие бейзлайна $MAP@10 \geq 0.074921$ **(6 баллов)**
- Код сервиса соответствует критериям читаемости и воспроизводимости **(3 балла)**

## Imports

In [1]:
import warnings

warnings.filterwarnings('ignore')

import pandas as pd
import time
import numpy as np
from pprint import pprint
import dill
from implicit.als import AlternatingLeastSquares

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import PopularModel, RandomModel, ImplicitALSWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel
import nmslib

import optuna

from pathlib import Path

from lightfm import LightFM

Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


## Подгружаем данные

In [2]:
DATA_PATH = Path("data_original")

In [3]:
users = pd.read_csv(DATA_PATH / 'users.csv')
items = pd.read_csv(DATA_PATH / 'items.csv')
interactions = pd.read_csv(DATA_PATH / 'interactions.csv')

In [4]:
users

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
...,...,...,...,...,...
840192,339025,age_65_inf,income_0_20,Ж,0
840193,983617,age_18_24,income_20_40,Ж,1
840194,251008,NaN,NaN,NaN,0
840195,590706,NaN,NaN,Ж,0


In [5]:
items

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15958,6443,series,Полярный круг,Arctic Circle,2018.0,"драмы, триллеры, криминал","Финляндия, Германия",NaN,16.0,NaN,Ханну Салонен,"Иина Куустонен, Максимилиан Брюкнер, Пихла Вии...","Во время погони за браконьерами по лесу, сотру...","убийство, вирус, расследование преступления, н..."
15959,2367,series,Надежда,NaN,2020.0,"драмы, боевики",Россия,0.0,18.0,NaN,Елена Хазанова,"Виктория Исакова, Александр Кузьмин, Алексей М...",Оригинальный киносериал от создателей «Бывших»...,"Надежда, 2020, Россия"
15960,10632,series,Сговор,Hassel,2017.0,"драмы, триллеры, криминал",Россия,0.0,18.0,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Р...",Криминальная драма по мотивам романов о шведск...,"Сговор, 2017, Россия"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родд...",Семнадцатилетний Дэмиен мечтает вырваться за п...,"Среди, камней, 2019, Россия"


In [6]:
interactions

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
...,...,...,...,...,...
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0


## Подготовка данных

In [7]:
Columns.Datetime = 'last_watch_dt'
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [8]:
max_date = interactions[Columns.Datetime].max()

In [9]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [10]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985269, 6)
test: (490982, 6)


In [11]:
train.drop(train.query("total_dur < 300").index, inplace=True)

In [12]:
# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

In [13]:
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

## Подготовка фичей

### Пользовательские фичи

In [14]:
# Сколько в % незаполненных полей у Пользователя
users.isnull().sum()/len(users)

user_id     0.000000
age         0.016776
income      0.017586
sex         0.016462
kids_flg    0.000000
dtype: float64

In [15]:
# % не большой => можно заменить пустые значения
users.fillna('Unknown', inplace=True)

In [16]:
# оставляем у df users только тех, кто попал в train
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()

In [17]:
users

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
5,1037719,age_45_54,income_60_90,М,0
...,...,...,...,...,...
840184,529394,age_25_34,income_40_60,Ж,0
840186,80113,age_25_34,income_40_60,Ж,0
840188,312839,age_65_inf,income_60_90,Ж,0
840189,191349,age_45_54,income_40_60,М,1


In [18]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex


### Фичи фильмов

In [19]:
# Сколько в % незаполненных полей у Item
items.isnull().sum()/len(items)

item_id         0.000000
content_type    0.000000
title           0.000000
title_orig      0.297250
release_year    0.006139
genres          0.000000
countries       0.002318
for_kids        0.964543
age_rating      0.000125
studios         0.933283
directors       0.094531
actors          0.164067
description     0.000125
keywords        0.026499
dtype: float64

In [20]:
# оставляем у df users только тех, кто попал в train
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

### Жанр

In [21]:
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


### Содержание

In [22]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
content_feature.head()

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,7868,film,content_type
4,16268,film,content_type


### Режисер

In [23]:
items["director"] = items["directors"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
director_feature = items[["item_id", "director"]].explode("director")
director_feature.columns = ["id", "value"]
director_feature["feature"] = "director"
director_feature.head()

,id,value,feature
0,10711,педро альмодовар,director
1,2508,скот армстронг,director
2,10716,адам п. калтраро,director
3,7868,эндрю хэй,director
4,16268,виктор садовский,director


### Страна

In [24]:
items["country"] = items["countries"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
country_feature = items[["item_id", "country"]].explode("country")
country_feature.columns = ["id", "value"]
country_feature["feature"] = "country"
country_feature.head()

,id,value,feature
0,10711,испания,country
1,2508,сша,country
2,10716,канада,country
3,7868,великобритания,country
4,16268,ссср,country


### Год Выпуска

In [25]:
year_feature = items.reindex(columns=[Columns.Item, "release_year"])
year_feature.columns = ["id", "value"]
year_feature["feature"] = "release_year"
year_feature.head()

,id,value,feature
0,10711,2002.0,release_year
1,2508,2014.0,release_year
2,10716,2011.0,release_year
3,7868,2015.0,release_year
4,16268,1978.0,release_year


In [26]:
items

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,genre,director,country
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...","[драмы, зарубежные, детективы, мелодрамы]",[педро альмодовар],[испания]
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео...","[зарубежные, приключения, комедии]",[скот армстронг],[сша]
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг...","[криминал, зарубежные, триллеры, боевики, коме...",[адам п. калтраро],[канада]
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю...","[драмы, зарубежные, мелодрамы]",[эндрю хэй],[великобритания]
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж...","[драмы, спорт, советские, мелодрамы]",[виктор садовский],[ссср]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15958,6443,series,Полярный круг,Arctic Circle,2018.0,"драмы, триллеры, криминал","Финляндия, Германия",NaN,16.0,NaN,Ханну Салонен,"Иина Куустонен, Максимилиан Брюкнер, Пихла Вии...","Во время погони за браконьерами по лесу, сотру...","убийство, вирус, расследование преступления, н...","[драмы, триллеры, криминал]",[ханну салонен],"[финляндия, германия]"
15959,2367,series,Надежда,NaN,2020.0,"драмы, боевики",Россия,0.0,18.0,NaN,Елена Хазанова,"Виктория Исакова, Александр Кузьмин, Алексей М...",Оригинальный киносериал от создателей «Бывших»...,"Надежда, 2020, Россия","[драмы, боевики]",[елена хазанова],[россия]
15960,10632,series,Сговор,Hassel,2017.0,"драмы, триллеры, криминал",Россия,0.0,18.0,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Р...",Криминальная драма по мотивам романов о шведск...,"Сговор, 2017, Россия","[драмы, триллеры, криминал]","[эшреф рейбрук, амир камдин, эрик эгер]",[россия]
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родд...",Семнадцатилетний Дэмиен мечтает вырваться за п...,"Среди, камней, 2019, Россия","[драмы, спорт, криминал]","[марк о’коннор, конор макмахон]",[россия]


In [27]:
# Объединяем фичи
item_features = pd.concat((genre_feature, content_feature, country_feature, year_feature, director_feature))
item_features

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre
...,...,...,...
15960,10632,амир камдин,director
15960,10632,эрик эгер,director
15961,4538,марк о’коннор,director
15961,4538,конор макмахон,director


## Метрики

In [28]:
metrics_name = {
    'Precision': Precision,
    'Recall': Recall,
    'MAP': MAP,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in range(1, 11):
        metrics[f'{metric_name}@{k}'] = metric(k=k)


pprint(metrics)

{'MAP@1': MAP(k=1, divide_by_k=False),
 'MAP@10': MAP(k=10, divide_by_k=False),
 'MAP@2': MAP(k=2, divide_by_k=False),
 'MAP@3': MAP(k=3, divide_by_k=False),
 'MAP@4': MAP(k=4, divide_by_k=False),
 'MAP@5': MAP(k=5, divide_by_k=False),
 'MAP@6': MAP(k=6, divide_by_k=False),
 'MAP@7': MAP(k=7, divide_by_k=False),
 'MAP@8': MAP(k=8, divide_by_k=False),
 'MAP@9': MAP(k=9, divide_by_k=False),
 'Precision@1': Precision(k=1),
 'Precision@10': Precision(k=10),
 'Precision@2': Precision(k=2),
 'Precision@3': Precision(k=3),
 'Precision@4': Precision(k=4),
 'Precision@5': Precision(k=5),
 'Precision@6': Precision(k=6),
 'Precision@7': Precision(k=7),
 'Precision@8': Precision(k=8),
 'Precision@9': Precision(k=9),
 'Recall@1': Recall(k=1),
 'Recall@10': Recall(k=10),
 'Recall@2': Recall(k=2),
 'Recall@3': Recall(k=3),
 'Recall@4': Recall(k=4),
 'Recall@5': Recall(k=5),
 'Recall@6': Recall(k=6),
 'Recall@7': Recall(k=7),
 'Recall@8': Recall(k=8),
 'Recall@9': Recall(k=9)}


## Подбор гиперпараметртов

In [36]:
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type", "director", "country", "release_year"],
)

TEST_USERS = test[Columns.User].unique()
results_opto = []


def objective(trial):
    # общие параметры
    K_RECOS = 10
    RANDOM_STATE = 42
    NUM_THREADS = 4
    N_FACTORS = trial.suggest_int("factors", 32, 64, 32, log=False)

    reco_model = trial.suggest_categorical("reco_model", ["ALS", "LightFM"])
    model_quality_opto = {"model": f"{reco_model}_{trial.number}"}

    if reco_model == "ALS":
        # гиперпараметры для ALS
        regularization = trial.suggest_float("regularization",
                                             1e-4,
                                             1.0,
                                             log=False)
        iterations = trial.suggest_int("iterations", 1, 6, 2, log=False)
        is_fitting_features = True
        # Инициализация ALS
        model = ImplicitALSWrapperModel(
            model=AlternatingLeastSquares(factors=N_FACTORS,
                                 random_state=RANDOM_STATE,
                                 num_threads=NUM_THREADS,
                                 regularization=regularization,
                                 iterations=iterations),
            fit_features_together=is_fitting_features,
        )
    else:
        # гиперпараметры для Lightfm
        N_EPOCHS = trial.suggest_int("epochs", 1, 50, 10, log=False)  # Lightfm
        USER_ALPHA = trial.suggest_float("user_alpha",
                                         1e-3,
                                         1.0,
                                         log=True)  # Lightfm
        ITEM_ALPHA = trial.suggest_float("item_alpha", 1e-3, 1.0,
                                         log=True)  # Lightfm
        LEARNING_RATE = trial.suggest_float("learning_rate",
                                            1e-4,
                                            1.0,
                                            log=True)  # Lightfm
        # Инициализация Lightfm
        model = LightFMWrapperModel(
            LightFM(
                no_components=N_FACTORS,
                loss=loss,
                random_state=RANDOM_STATE,
                learning_rate=LEARNING_RATE,
                user_alpha=USER_ALPHA,
                item_alpha=ITEM_ALPHA,
            ),
            epochs=N_EPOCHS,
            num_threads=NUM_THREADS,
        )

    # обучение модели
    model.fit(dataset)
    recos = model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )

    # Подсчет метрик
    metric_values = calc_metrics(metrics, recos, test, train)
    model_quality_opto.update(metric_values)
    results_opto.append(model_quality_opto)

    return metric_values.get('MAP@10') # максимизируемая метрика

In [ ]:
# запуск подбора гиперпараметров
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1)

pprint(f"Number of finished trials: {len(study.trials)}")
trial = study.best_trial
pprint(f"Best trial: {trial}")

## Approximate Nearest Neighbors

In [39]:
K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 4
N_FACTORS = 64
N_EPOCHS = 1 # Lightfm
USER_ALPHA = 0 # Lightfm
ITEM_ALPHA = 0 # Lightfm
LEARNING_RATE = 0.03 # Lightfm

In [40]:
model = LightFMWrapperModel(
            LightFM(
                no_components=N_FACTORS,
                loss='warp',
                random_state=RANDOM_STATE,
                learning_rate=LEARNING_RATE,
                user_alpha=USER_ALPHA,
                item_alpha=ITEM_ALPHA,
            ),
            epochs=N_EPOCHS,
            num_threads=NUM_THREADS,
        )

model.fit(dataset)

In [41]:
recos = model.recommend(
    users=TEST_USERS,
    dataset=dataset,
    k=K_RECOS,
    filter_viewed=True,
)

In [42]:
results = []
model_quality = {'model': 'LightFM'}
metric_values = calc_metrics(metrics, recos, test, train)
model_quality.update(metric_values)
results.append(model_quality)

df_quality = pd.DataFrame(results).T

df_quality.columns = df_quality.iloc[0]

df_quality.drop('model', inplace=True)

In [43]:
df_quality.style.highlight_max(color='lightgreen', axis=1)

model,LightFM
Precision@1,0.083043
Recall@1,0.042287
Precision@2,0.070506
Recall@2,0.070637
Precision@3,0.062113
Recall@3,0.091601
Precision@4,0.055932
Recall@4,0.108630
Precision@5,0.050513
Recall@5,0.121060


In [44]:
dill_file = Path().cwd().parent / 'service' / 'models'

with open(dill_file / 'LightFM_0.078294.dill', 'wb') as f:
    dill.dump(model, f)

In [45]:
dill_file = Path().cwd().parent / 'service' / 'data'

with open(dill_file / 'dataset_LightFM_0.078294.dill', 'wb') as f:
    dill.dump(dataset, f)

In [69]:
user_embeddings, item_embeddings = model.get_vectors(dataset)

In [70]:
user_embeddings.shape, item_embeddings.shape

((756562, 66), (14019, 66))

In [71]:
def augment_inner_product(factors):
    normed_factors = np.linalg.norm(factors, axis=1)
    max_norm = normed_factors.max()

    extra_dim = np.sqrt(max_norm ** 2 - normed_factors ** 2).reshape(-1, 1)
    augmented_factors = np.append(factors, extra_dim, axis=1)
    return max_norm, augmented_factors

In [72]:
print('pre shape: ', item_embeddings.shape)
max_norm, augmented_item_embeddings = augment_inner_product(item_embeddings)
augmented_item_embeddings.shape

pre shape:  (14019, 66)


(14019, 67)

In [73]:
extra_zero = np.zeros((user_embeddings.shape[0], 1))
augmented_user_embeddings = np.append(user_embeddings, extra_zero, axis=1)
augmented_user_embeddings.shape

(756562, 67)

In [56]:
user_id = 30
user_embeddings[user_id]

array([-1.34910004e+02,  1.00000000e+00,  3.72829586e-02, -2.29887888e-01,
        2.19721347e-02, -1.17607884e-01,  1.86465487e-01, -4.29087058e-02,
       -9.65012684e-02,  1.86485201e-01,  7.40239322e-02, -6.97900653e-02,
        9.24971402e-02, -7.78931379e-02, -2.50789046e-01,  3.92958410e-02,
        7.60998577e-02, -1.16163656e-01, -1.98737621e-01, -1.04761831e-01,
       -4.61863279e-02, -5.44709638e-02, -3.41998599e-03,  1.40651405e-01,
       -2.58379817e-01,  1.48677984e-02,  4.19342145e-02,  8.75925198e-02,
       -1.50252372e-01, -9.44809392e-02,  8.93273950e-02,  1.06742702e-01,
        4.64314707e-02,  7.10017085e-02,  8.84427354e-02, -1.77015409e-01,
        2.45146394e-01, -2.53994107e-01,  1.59706324e-01,  1.81685910e-02,
       -1.80056110e-01,  1.28522277e-01,  2.57659584e-01,  1.45844501e-02,
        1.19970128e-01,  3.72234024e-02, -2.02342212e-01,  4.92374375e-02,
        2.48806834e-01, -1.26030415e-01, -3.70325483e-02,  1.50772557e-01,
        4.21895273e-03, -

In [43]:
augmented_user_embeddings[user_id]

array([-2.25044174e+02,  1.00000000e+00,  2.76991814e-01,  3.94240022e-01,
        8.08445066e-02,  1.82592094e-01, -2.43438885e-01,  1.93806931e-01,
        2.65002817e-01,  3.94734919e-01,  3.23766947e-01,  2.76526213e-01,
        8.90274867e-02, -1.47107691e-01,  1.14467442e-01,  3.59575599e-02,
       -3.20784628e-01, -2.19055980e-01,  1.51054800e-01, -4.06866595e-02,
       -2.61475056e-01,  3.26546431e-01, -3.56499314e-01, -1.09140053e-01,
        3.72507572e-01, -2.71710515e-01,  3.04252625e-01,  2.92776406e-01,
       -1.57168627e-01, -1.99281722e-01, -8.09721947e-02,  2.42758915e-01,
       -3.75919640e-02,  7.07380101e-02,  0.00000000e+00])

In [74]:
item_id = 0
item_embeddings[item_id]

array([ 1.        ,  1.94914997, -0.23860815,  0.03788533,  0.25774899,
       -0.74438763,  0.69889945, -0.67908454, -0.68033189,  0.52302718,
       -0.1655457 , -1.05006051, -0.42892846, -0.2613495 , -0.27979758,
       -0.1573247 ,  0.12265188, -0.54494703,  0.17335984, -0.15847054,
       -0.01530413, -0.34174129,  0.52183533,  0.14352471, -0.50889617,
       -0.79903305, -0.34760365, -0.14387724, -0.12861755, -0.21334469,
        0.68404454,  0.02518467,  0.64104861, -0.48303077,  0.82251263,
       -0.45924279,  0.234642  ,  0.00584684,  0.32230985,  0.13046621,
       -0.55888671, -0.35819465,  0.26253027, -0.58298248,  0.36183062,
       -0.29011068, -0.53527212,  0.07575223, -0.07332505, -0.56368858,
       -0.55525213, -0.00949526,  0.54710573, -0.52100545, -0.60292774,
       -0.39715344, -0.06575568, -0.07951353,  0.53371233,  0.59348917,
       -0.47663444, -0.81328684, -0.31239626, -0.00675351,  0.30547792,
       -0.00500108])

In [46]:
augmented_item_embeddings[item_id]

array([ 1.        ,  2.17175078,  0.34701797,  1.19930804,  0.22779027,
        0.03909439, -1.08578384,  0.9331314 ,  1.14171183,  0.42676285,
        0.78357077,  0.81151915, -0.27156514, -0.21403486,  1.12571526,
       -0.85592711, -0.60585177, -0.84999949,  1.61589718, -0.15849388,
       -1.39642692,  0.99047184, -0.94934148, -0.44196448,  0.52439356,
       -0.6769855 ,  0.67952645, -0.68347728, -0.39412886, -1.4741863 ,
       -0.56587529,  1.03477836, -1.17399275,  0.99339014,  3.60761326])

In [167]:
# Set index parameters
# These are the most important ones
M = 48
efC = 100

# Number of neighbors
K=10

# Space name should correspond to the space name
# used for brute-force search
space_name='negdotprod'

num_threads = 4
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}
print('Index-time parameters', index_time_params)

Index-time parameters {'M': 48, 'indexThreadQty': 4, 'efConstruction': 100, 'post': 0}


In [168]:

# Intitialize the library, specify the space, the type of the vector and add data points
index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR)
index.addDataPointBatch(augmented_item_embeddings)

14019

In [169]:
# Create an index
start = time.time()
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC}
index.createIndex(index_time_params)
end = time.time()
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 48, 'indexThreadQty': 4, 'efConstruction': 100}
Indexing time = 0.481232


In [170]:
# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params)

Setting query-time parameters {'efSearch': 100}


In [88]:
user_index = np.where(dataset.user_id_map.external_ids == 1032142)[0][0]
query_matrix = augmented_user_embeddings[user_index]

In [79]:
# Querying
query_qty = query_matrix.shape[0]
start = time.time()
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)
end = time.time()
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' %
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty))

kNN time total=0.023637 (sec), per query=0.000024 (sec), per query adjusted for thread number=0.000095 (sec)


In [89]:
nbrs = index.knnQueryBatch([query_matrix], k = K, num_threads = num_threads)

In [38]:
# получение рекомендации по конкретному пользователю
def recommend_nmslib(user_id, K=10, num_threads=4):
    user_index = np.where(dataset.user_id_map.external_ids == user_id)[0][0]
    query_matrix = augmented_user_embeddings[user_index]
    nbrs = index.knnQueryBatch([query_matrix], k = K, num_threads = num_threads)
    labels = dataset.item_id_map.external_ids[nbrs[0][0]]
    distances = nbrs[0][1]
    return labels, distances

In [172]:
res = []
for user in TEST_USERS[:]:
    labels, distances = recommend_nmslib(user)
    labels_u = np.zeros((len(labels), 4), dtype=int)
    labels_u[:, 0] = user

    # проставляем порядок треков
    labels_u[:, 3] = range(len(labels))
    # присваем треки в обратном порядке, чтобы самый свежий терк был в начале
    labels_u[:, 1] = labels
    labels_u[:, 2] = distances
    res.extend(labels_u)

In [173]:
recos_ = pd.DataFrame(res, columns=['user_id', 'item_id','score','rank'])

In [174]:
# подсчет метрик
results = []
model_quality = {'model': 'nmslib'}
metric_values = calc_metrics(metrics, recos_, test, train)
model_quality.update(metric_values)
results.append(model_quality)

df_quality = pd.DataFrame(results).T

df_quality.columns = df_quality.iloc[0]

df_quality.drop('model', inplace=True)

In [175]:
df_quality.style.highlight_max(color='lightgreen', axis=1)

model,nmslib
Precision@1,0.109757
Recall@1,0.055766
Precision@2,0.076285
Recall@2,0.076216
Precision@3,0.063157
Recall@3,0.093738
Precision@4,0.054932
Recall@4,0.107254
Precision@5,0.049375
Recall@5,0.118995


In [59]:
def recommend_all(query_factors, index_factors, topn=10):
    output = query_factors.dot(index_factors.T)
    argpartition_indices = np.argpartition(output, -topn)[:, -topn:]

    x_indices = np.repeat(np.arange(output.shape[0]), topn)
    y_indices = argpartition_indices.flatten()
    top_value = output[x_indices, y_indices].reshape(output.shape[0], topn)
    top_indices = np.argsort(top_value)[:, ::-1]

    y_indices = top_indices.flatten()
    top_indices = argpartition_indices[x_indices, y_indices]
    labels = top_indices.reshape(-1, topn)
    distances = output[x_indices, top_indices].reshape(-1, topn)
    return labels, distances
